In [27]:
import pandas as pd
from datetime import datetime
import numpy as np
import json
import glob
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
list_of_file_names = glob.glob("./zip_2/*.csv")

In [5]:
read_data_list_2 = []
fail_list = []
for i, data in enumerate(list_of_file_names):
    try:
        data = pd.read_csv(data)
        #data = unicode(data, errors='replace')
        read_data_list_2.append(data)
    except UnicodeDecodeError:
        fail_list.append(i)
#read_data_list
#fail_list
list_of_file_names_org = list_of_file_names

In [6]:
fail_list

[3]

In [7]:
for x in fail_list:
    del list_of_file_names[x]

In [8]:
# build loop for data we can work with. Only dates and zip
def build_useful_df(data_list):
    useful_df = []
    for dataset in range(len(data_list)):
        data = read_data_list_2[dataset]
        region_name = data['RegionName']
        new_df = data.select_dtypes(include=['float64'])
        new_df.insert(loc=0, column='RegionName', value=region_name) 
        new_df = new_df.sort_values('RegionName',ascending=False)
        new_df = new_df.set_index('RegionName')
        useful_df.append(new_df)
    return useful_df

In [9]:
useful_df = build_useful_df(read_data_list_2)
useful_df[3].head()

,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
RegionName,,,,,,,,,,,,,,,,,,,,,
99801,5.001480,12.059667,9.721639,12.823805,12.067908,6.735467,12.817048,5.351705,7.022677,11.403385,...,8.149935,7.839226,8.340472,11.016614,13.717871,12.825190,12.298428,13.377564,11.830469,10.323291
99709,13.332107,11.447750,20.230705,17.569812,9.501848,16.866147,25.336712,14.339757,14.388552,11.076664,...,15.020218,15.179652,9.608745,15.497014,13.444893,21.787553,14.004356,17.560603,14.040229,12.349473
99705,14.238275,11.455669,21.931982,19.715816,13.152595,16.048516,16.539511,13.047547,15.500396,18.073011,...,17.252181,14.419267,13.725248,14.145342,15.184207,21.538185,15.127410,18.841846,19.314115,18.508919
99701,12.502596,3.868248,11.226388,8.029469,11.557336,13.020347,6.255375,7.774582,10.831500,12.234200,...,11.990818,11.121010,13.573817,14.626545,17.361696,14.490435,12.286012,12.098577,12.474354,16.429898
99669,12.342694,9.547881,9.399721,11.789248,13.575353,8.824402,11.448679,12.713187,12.052649,9.105020,...,8.814888,10.137303,13.932264,12.073197,9.835698,14.488406,11.190449,15.390050,11.528148,13.687983


In [ ]:
# concat all dfs into one multi-index. I am not currently using this
def df_build_dataframe(df_list, beginning, end):
    length = len(df_list[beginning:end])
    keys = []
    values = []
    final = []
    for x in range(length):
        y = str(x)
        keys.append(y)
    for df in df_list[beginning:end]:
        values.append(df)
    dictionary = dict(zip(keys, values))
    df = pd.concat(dictionary)
    return df
single_df = df_build_dataframe(useful_df, 3, 71)

In [11]:
# Building a list of all dataframes that work seamlessly with function. Tested through trial and error to get these 
# values
good_range = list(range(72))
good_range.remove(0)
good_range.remove(1)
good_range.remove(2)

In [12]:
# Building the features for the model. The ones commented out I want to work on after I run the initial model
# They will have more parameters for instance, "did this zipcode have a big swing and gain value?"
# I also want to add some of those rolling type of features like "mean, std, min, max, swing, change, for last three months"
# Also listing any files that don't fit the date parameters in 'bugs'
def build_feature_list(good_range, df_list):
    year_list = ['2011', '2012', '2013', '2014', '2015', '2016']
    feature_df_list = []
    bugs = []
    for num in good_range:
        df = df_list[num]
        try:
            df = df.loc[:, '2011-01':'2016-12']
            features = pd.DataFrame()
            for i, year in enumerate(year_list):
                mean = df.loc[:, year + '-01': year + '-12'].mean(axis=1)
                features[year + '_mean'] = mean
                std = df.loc[:, year + '-01': year + '-12'].std(axis=1)
                features[year + '_std'] = std
                mn = df.loc[:, year + '-01': year + '-12'].min(axis=1)
                features[year + '_min'] = mn
                mx = df.loc[:, year + '-01': year + '-12'].max(axis=1)
                features[year + '_max'] = mx
                features[year + '_swing'] = mx - mn
                change = df[year + '-12'] - df[year + '-01']
                features[year + '_change'] = change
                if i > 0:
                    yoy = features[year + '_mean'] / df.loc[:, year_list[i - 1] + '-01': year_list[i - 1] + '-12'].mean(axis=1)
                    features[year + '_yoy'] = yoy
                    #features[year + '_gain'] = np.where(features[year + '_yoy']>1, 1, 0)
                    #big swing and gain, big swing and loss, big swing and big gain, big swing and big loss
            feature_df_list.append(features)
        except:
            bugs.append(num)
    return feature_df_list, bugs

In [13]:
feature_df_list, bug_list = build_feature_list(good_range, useful_df)

In [14]:
feature_df_list[0].head()

,2011_mean,2011_std,2011_min,2011_max,2011_swing,2011_change,2012_mean,2012_std,2012_min,2012_max,...,2015_swing,2015_change,2015_yoy,2016_mean,2016_std,2016_min,2016_max,2016_swing,2016_change,2016_yoy
RegionName,,,,,,,,,,,,,,,,,,,,,
99801,9.310209,2.944113,3.948526,12.817048,8.868522,-2.369770,8.136485,3.436430,3.786004,14.723430,...,10.922510,4.423798,1.147612,10.068442,3.062214,5.088322,15.190752,10.102429,-2.221789,0.815729
99709,14.946397,4.360379,9.436303,25.336712,15.900409,5.824062,13.842236,2.040111,10.402749,16.453952,...,7.886666,-3.434515,0.890816,14.867648,1.965726,10.527746,17.673090,7.145344,-1.084411,1.052896
99705,14.737880,3.007675,9.272206,20.583079,11.310873,-0.965515,14.121202,2.229902,9.937863,18.068721,...,9.822462,0.918776,0.778620,13.001790,3.032479,9.026161,19.540471,10.514310,-7.450660,0.959551
99701,10.091596,2.632644,6.226229,13.020347,6.794118,1.073704,13.299571,4.214313,8.873875,21.746954,...,9.228073,-1.347940,0.930134,12.777948,2.078412,10.289255,16.890567,6.601312,-2.607957,0.805753
99669,10.798498,1.890953,8.234307,13.575353,5.341046,-4.060254,9.651472,1.428610,7.102004,11.841231,...,8.172776,4.037065,1.052458,13.092470,3.342978,8.640817,21.497990,12.857173,-1.582450,1.163023


In [15]:
bug_list

[34, 45, 48, 52, 59, 60, 61, 62, 63, 64, 65, 67, 68, 70, 71]

In [16]:
# Merging all dfs into one. Merging on indez which is zipcode. I was hoping for inner join but it looks like
# There are many zipcodes that only exist in certain dfs. Im hoping that reducing them to metro areas will fix this
def merge_dataframes(feature_df_list):
    df_1 = feature_df_list[0]
    for df in feature_df_list[1:]:
        df_1 = pd.merge(df_1, df, left_index=True, right_index=True, how='outer')
    return df_1

In [19]:
# Are NaN's due to outer merge?
X = merge_dataframes(feature_df_list)
X.tail()

,2011_mean_x,2011_std_x,2011_min_x,2011_max_x,2011_swing_x,2011_change_x,2012_mean_x,2012_std_x,2012_min_x,2012_max_x,...,2015_swing_y,2015_change_y,2015_yoy_y,2016_mean_y,2016_std_y,2016_min_y,2016_max_y,2016_swing_y,2016_change_y,2016_yoy_y
RegionName,,,,,,,,,,,,,,,,,,,,,
99709,14.946397,4.360379,9.436303,25.336712,15.900409,5.824062,13.842236,2.040111,10.402749,16.453952,...,0.046971,0.046971,-0.890296,0.048043,0.014847,0.027583,0.071696,0.044113,0.006359,4.154441
99712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.059971,0.055702,0.333645,0.058742,0.013431,0.030547,0.077129,0.046582,-0.017640,-14.320149
99801,9.310209,2.944113,3.948526,12.817048,8.868522,-2.369770,8.136485,3.436430,3.786004,14.723430,...,0.075475,-0.075475,0.050715,-0.022789,0.024616,-0.047337,0.012188,0.059525,0.052582,-14.033968
99835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.002694,0.001342,5.585394,0.012510,0.017565,-0.004944,0.046459,0.051403,0.048261,-5.974465


In [29]:
# Shape of input array
X = np.asarray(X)
X.shape

(17611, 2214)

In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(2214,)))
model.add(Dropout(.25))
model.add(Dense(128))
model.add(Dropout(.1))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])